## Custom GPT Model - Shakespear Dataset

In [ ]:
#This is for FIRST RUN ONLY to download shakespear data
'''
import wget
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
shakes = wget.download(url)
shakes

In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("Length of dataset: " + str(len(text)))

Length of dataset: 1115394


In [4]:
print(text[:500]) #Print first 500 characters

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [5]:
# Unique characters in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### 1. Encoder - Decoder

In [6]:
#Create a mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] #Encoder: Take string input and produce integer
decode = lambda l: ''.join([itos[i] for i in l]) #Encoder: Take list of integers and produce string

print(encode("Hello meesam"))
print(decode(encode("Hello meesam")))
#It should be noted that our tokenizer is CHARACTER LEVEL tokenizer
#GPT and GOOGle use BPE (byte pair encoding) or other schemes - subword basis

[20, 43, 50, 50, 53, 1, 51, 43, 43, 57, 39, 51]
Hello meesam


### 2. Torch tensors

In [7]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:500])

ModuleNotFoundError: No module named 'torch'

### 3. Train / validation split

In [ ]:
n = int(0.9*len(data))  #90% is for training rest validation
train_data = data[:n]
val_data = data[n:]

In [ ]:
#Let's do training on data CHUNKS
block_size = 8
train_data[:block_size + 1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"When input is {context} then target is : {target}")

### 4. Batch of Input Tensors

In [ ]:
torch.manual_seed(1337)
batch_size = 4              # number of independent sequences to be processed in parallel
block_size = 8              # maximum context length of predictions
def get_batch(split):
    #Generate small batch of inputs of data input x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
xb, yb = get_batch('train')
print("inputs: ")
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)
print('-------------')
for b in range(batch_size):          #Batch dimension
    for t in range(block_size):      #Time dimension
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target is: {target}")


In [ ]:
print(xb)

### 5. Feed to BigramLanguageModel Neural Network (NN)

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #Each token reads off directly reads off the logits for the next token lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        #idx and targets are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx)
        return logits

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
